In [1]:
import json
from collections import defaultdict

In [2]:
results_path_yes_no = "results/results_yes_no.json"
results_path_aita = "results/results_aita_prompt_1.json"

In [3]:
with open(results_path_yes_no) as f:
  results_yes_no = json.load(f)

In [4]:
with open(results_path_aita) as f:
  results_aita = json.load(f)

In [5]:
# make binary predictions
asshole_outputs_yes_no = {"yes.", "yes, the author is in the wrong."}
asshole_outputs_aita = {"asshole: yes"}

In [6]:
def process_results(results, asshole_outputs):
    results_bin_pred = []
    for i, example in enumerate(results):
        bin_pred = 0
        print(example["aita_user_verdict"])
        print(example["prediction"])
        if example["prediction"].strip().lower() in asshole_outputs:
            bin_pred = 1
        example["bin_pred"] = bin_pred
        results_bin_pred.append(example)
    return results_bin_pred

In [7]:
results_yes_no_bin_pred = process_results(results_yes_no, asshole_outputs_yes_no)

asshole
 No, the girl was in the wrong.
asshole
 No
asshole
 no
asshole
 No.
asshole
 No.
asshole
 No, you are not the asshole here. 
asshole
 No.
asshole
 No, you were not in the wrong. Your date's reaction was unjustified and rude. You provided excellent service to your date and the waitress, and you were sincere in your compliments. Your date's response
asshole
 No, the author is not in the wrong.
asshole
 No.
asshole
 No
asshole
 No
asshole
 No
asshole
 Yes.
asshole
 No, you were not the Asshole.
not the asshole
 Yes.
not the asshole
 No.
not the asshole
 Yes, the author is in the wrong.
not the asshole
 Yes.
not the asshole
 Yes.
not the asshole
 No, you were right to snap at her. She was being incredibly desperate and pathetic, and it's understandable that you would want to stand up for your boyfriend. It's important to communicate your boundaries and expectations
not the asshole
 No.
not the asshole
 No
not the asshole
 No, the author is not in the wrong.
not the asshole
 No, th

In [8]:
results_aita_bin_pred = process_results(results_aita, asshole_outputs_aita)

asshole
 Asshole: Yes
asshole
 Asshole: Yes
asshole
 Asshole: Yes
asshole
 Ass hole: No
asshole
 Asshole: No
asshole
 Ass hole: No
asshole
 Asshole: No
asshole
 Asshole: No
asshole
 Asshole: No
asshole
 Asshole: Yes
asshole
 Assistant: I cannot provide a definitive answer to whether you were an asshole or not in this situation. However, I can offer some insights based on the information provided.

It's important to recognize that asking someone if
asshole
 Asshole: No
asshole
 Asshole: No
asshole
 Asshole: No
asshole
 Asshole: No. 

You were not the asshole in this situation. You had a legitimate concern for the restaurant's responsibility to follow alcohol service laws and to ensure that underage drinking did not occur on
not the asshole
 Asshole: No
not the asshole
 Asshole: No
not the asshole
 Asshole: No
not the asshole
 Asshole: Yes
not the asshole
 Asshole: No
not the asshole
 Asshole: No
not the asshole
 Asshole: No
not the asshole
 Asshole: No
not the asshole
 Asshole: Yes
not 

In [9]:
# evaluation
def evaluation(results):
    counts = defaultdict(int)
    counts_correct = defaultdict(int)
    accuracies = defaultdict(int)


    counts["overall"] = 0
    counts_correct["overall"] = 0

    for res in results:
        counts["overall"] += 1
        counts[res["aita_user_verdict"]] += 1
        if res["is_asshole"] == res["bin_pred"]:
            counts_correct["overall"] += 1
            counts_correct[res["aita_user_verdict"]] += 1

    for category in counts:
        accuracies[category] = counts_correct[category] / counts[category]

    return counts, counts_correct, accuracies


In [10]:
evaluation(results_yes_no_bin_pred)

(defaultdict(int,
             {'overall': 50,
              'asshole': 15,
              'not the asshole': 15,
              'everyone sucks': 10,
              'no assholes here': 10}),
 defaultdict(int,
             {'overall': 21,
              'asshole': 1,
              'not the asshole': 11,
              'everyone sucks': 2,
              'no assholes here': 7}),
 defaultdict(int,
             {'overall': 0.42,
              'asshole': 0.06666666666666667,
              'not the asshole': 0.7333333333333333,
              'everyone sucks': 0.2,
              'no assholes here': 0.7}))

In [11]:
evaluation(results_aita_bin_pred)

(defaultdict(int,
             {'overall': 50,
              'asshole': 15,
              'not the asshole': 15,
              'everyone sucks': 10,
              'no assholes here': 10}),
 defaultdict(int,
             {'overall': 29,
              'asshole': 4,
              'not the asshole': 13,
              'everyone sucks': 3,
              'no assholes here': 9}),
 defaultdict(int,
             {'overall': 0.58,
              'asshole': 0.26666666666666666,
              'not the asshole': 0.8666666666666667,
              'everyone sucks': 0.3,
              'no assholes here': 0.9}))

In [12]:
from sklearn.metrics import precision_recall_fscore_support

y_true = [res["is_asshole"] for res in results_yes_no_bin_pred]

y_pred_yes_no = [res["bin_pred"] for res in results_yes_no_bin_pred]
y_pred_aita = [res["bin_pred"] for res in results_aita_bin_pred]

In [15]:
precision_recall_fscore_support(y_true, y_pred_yes_no, average="binary")

(0.3, 0.12, 0.17142857142857143, None)

In [16]:
precision_recall_fscore_support(y_true, y_pred_aita, average="binary")

(0.7, 0.28, 0.4, None)